In [3]:
#I want to get to know gradient boosting methods (in particular, the xgboost library) and i am also currently in barbados.
#Import libraries:
import numpy as np
import pandas as pd
import xgboost as xgb
import time
#load data:
train = pd.read_csv("train.csv")
target = train['target']
#drop targets & (unique row) IDs from training data
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
IDs = test['ID'].values
test = test.drop(['ID'],axis=1)

# PREPROCESSING

In [4]:
#impute both numerical & categorical features a la
#http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
xtrain = DataFrameImputer().fit_transform( train )
xtest = DataFrameImputer().fit_transform( test )

In [6]:
#factorize categorical columns:
for column in xtrain:
    if xtrain[column].dtype == 'O':
#         print pd.factorize(xtrain[column])
        xtrain[column] = pd.factorize(xtrain[column])[0]
    
for column in xtest:
    if xtest[column].dtype == 'O':
#         print pd.factorize(xtrain[column])
        xtest[column] = pd.factorize(xtest[column])[0]

Next up: scaling/transforms/get_dummies/dimensionality reduction

# GRADIENT BOOSTING & CROSS VALIDATION

In [8]:
#check this out: http://xgboost.readthedocs.org/en/latest/model.html
from sklearn.cross_validation import KFold, train_test_split
X = xtrain.values
y = target.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1 , random_state=0)

/Users/hugobowne-anderson/repos/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
# Early-stopping
#http://xgboost.readthedocs.org/en/latest/python/python_intro.html#early-stopping
#Also see https://github.com/dmlc/xgboost/blob/master/demo/guide-python/sklearn_examples.py (Jamie Hall et al.)
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="logloss",
        eval_set=[(X_test, y_test)])

Will train until validation_0 error hasn't decreased in 50 rounds.
[0]	validation_0-logloss:0.660512
[1]	validation_0-logloss:0.633833
[2]	validation_0-logloss:0.611452
[3]	validation_0-logloss:0.592876
[4]	validation_0-logloss:0.577408
[5]	validation_0-logloss:0.564382
[6]	validation_0-logloss:0.553405
[7]	validation_0-logloss:0.543860
[8]	validation_0-logloss:0.536023
[9]	validation_0-logloss:0.528992
[10]	validation_0-logloss:0.523194
[11]	validation_0-logloss:0.518142
[12]	validation_0-logloss:0.514029
[13]	validation_0-logloss:0.510511
[14]	validation_0-logloss:0.507166
[15]	validation_0-logloss:0.504451
[16]	validation_0-logloss:0.502286
[17]	validation_0-logloss:0.500022
[18]	validation_0-logloss:0.498165
[19]	validation_0-logloss:0.496556
[20]	validation_0-logloss:0.495312
[21]	validation_0-logloss:0.493927
[22]	validation_0-logloss:0.492970
[23]	validation_0-logloss:0.491965
[24]	validation_0-logloss:0.491137
[25]	validation_0-logloss:0.490338
[26]	validation_0-logloss:0.48966

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [14]:
preds = clf.predict_proba(xtest.values, ntree_limit=clf.best_iteration)[:,1]

In [16]:
import csv
predictions_file = open("xgboost_predictions.csv", "w")
open_file_object = csv.writer(predictions_file)
open_file_object.writerow(["ID", "PredictedProb"])
open_file_object.writerows(zip(IDs, preds))
predictions_file.close()

In [17]:
#also check this out:
#https://www.kaggle.com/yuhaichina/bnp-paribas-cardif-claims-management/extratreesclassifier-score-0-45911/discussion